In [3]:
import cv2
import numpy as np

# Load YOLO
cfg = r"C:\Users\Nagendran\Downloads\darknet\darknet-master\cfg\yolov3.cfg"
weights = r"C:\Hope_Learning\Deep_learning\Supervised_learning\Object Detection\yolo\yolov3.weights"
net = cv2.dnn.readNet(weights, cfg)

# Load class labels
coconames = r"C:\Users\Nagendran\Downloads\darknet\darknet-master\data\coco.names"
with open(coconames, "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Get layer names and output layers
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Initialize the webcam
cap = cv2.VideoCapture(0)  # 0 for default webcam, or change to 1, 2, etc. for external cameras

# Loop through frames
while True:
    # Capture each frame
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame to 416x416 for YOLO
    height, width, channels = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)

    # Perform forward pass
    outs = net.forward(output_layers)

    # Process detections
    class_ids, confidences, boxes = [], [], []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5:
                center_x, center_y = int(detection[0] * width), int(detection[1] * height)
                w, h = int(detection[2] * width), int(detection[3] * height)
                x, y = int(center_x - w / 2), int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply non-max suppression to avoid multiple boxes for the same object
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.4, 0.3)
    for i in indexes.flatten():
        x, y, w, h = boxes[i]
        label = f"{classes[class_ids[i]]}: {confidences[i]:.2f}"
        color = (0, 255, 0)  # Green color for bounding box
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # Show the live video with detections
    cv2.imshow("Live Detection", frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()
